In [2]:
import pyLDAvis
import pyLDAvis.sklearn

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

pyLDAvis.enable_notebook()

In [22]:
# we want a list of raw text from the files
# zero-carbon-bill/input/*.txt

from glob import glob

raw_texts = []

for file in glob("zero-carbon-bill/input/*.json"):
    with open(file) as f:
        contents = f.read()
    raw_texts.append(contents)
    
print(len(raw_texts))
print(raw_texts[1])

filenames = glob("./zero-carbon-bill/input/*.json")

599
No zero emissions, no climate change legislation without a public referendum to give a clear mandate. The current government does

not have such a mandate.



In [29]:
# Build a CountVectorizer

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Zā]{3,}\b',
                                max_df = 0.5, 
                                min_df = 2,
                                input = 'filename')

dtm_tf = tf_vectorizer.fit_transform(filenames)
print(dtm_tf.shape)

(599, 2468)


In [30]:
# Build a TFIDFVectorizer
# to compare shape

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(filenames)
print(dtm_tfidf.shape)

(599, 2468)


/Users/merrin/dia/youth_voice.venv/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


In [31]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [32]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/Users/merrin/dia/youth_voice.venv/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.051031 -0.033959       1        1  14.002166
11     0.032618 -0.072886       2        1  10.370021
0      0.094419 -0.019237       3        1   9.569104
14     0.037905 -0.036275       4        1   7.145969
16     0.035245 -0.121306       5        1   6.528866
6      0.005470  0.026185       6        1   5.660767
10    -0.016176 -0.082966       7        1   4.927155
19    -0.035532  0.120585       8        1   4.752544
18    -0.127726  0.042996       9        1   4.471759
5     -0.185652 -0.018930      10        1   4.401516
12     0.000024 -0.030171      11        1   3.973566
8     -0.026183 -0.022634      12        1   3.856524
3      0.006931  0.014678      13        1   3.689875
4      0.122989  0.098757      14        1   3.262117
17     0.045640 -0.026690      15        1   3.130085
9     -0.007409 -0.022355      16        1   2.832851
2     -0.085360 -0.022297      17        1   2.141987
13     0.029888  0.099335      18        1   1.930733
7      0.026596  0.006383      19        1   1.831835
1     -0.004716  0.100787      20        1   1.520559, topic_info=     Category        Freq        Term       Total  loglift  logprob
340   Default  308.000000     climate  308.000000  30.0000  30.0000
306   Default  305.000000      change  305.000000  29.0000  29.0000
941   Default  150.000000  government  150.000000  28.0000  28.0000
904   Default   85.000000      future   85.000000  27.0000  27.0000
1435  Default  176.000000        need  176.000000  26.0000  26.0000
...       ...         ...         ...         ...      ...      ...
1070  Topic20    1.520482      impact   34.897178   1.0527  -5.3355
2206  Topic20    1.520482         tax   35.614843   1.0324  -5.3355
2467  Topic20    1.520482        zero  108.232256  -0.0792  -5.3355
927   Topic20    1.520482       given   30.224106   1.1965  -5.3355
1807  Topic20    1.520482      reduce   60.584279   0.5011  -5.3355

[1383 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
1         1  0.155065  able
1         2  0.155065  able
1         3  0.077532  able
1         6  0.038766  able
1         8  0.077532  able
...     ...       ...   ...
2467     15  0.009239  zero
2467     16  0.018479  zero
2467     17  0.009239  zero
2467     19  0.018479  zero
2467     20  0.018479  zero

[4270 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 12, 1, 15, 17, 7, 11, 20, 19, 6, 13, 9, 4, 5, 18, 10, 3, 14, 8, 2])

In [33]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/Users/merrin/dia/youth_voice.venv/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.256770 -0.032969       1        1  19.670636
17    -0.020506  0.144340       2        1   9.951973
2      0.059115  0.105622       3        1   9.466542
16     0.009834 -0.024697       4        1   6.522953
11     0.007762  0.014120       5        1   4.572459
7     -0.002458  0.014130       6        1   4.561828
15    -0.006948 -0.007432       7        1   4.045075
5     -0.015185 -0.037459       8        1   3.975500
0     -0.021377 -0.016517       9        1   3.809772
14    -0.022268 -0.016104      10        1   3.758678
6     -0.053946 -0.013025      11        1   3.679715
12    -0.030801 -0.024005      12        1   3.260475
4     -0.035253 -0.020376      13        1   3.143370
9     -0.023008 -0.011987      14        1   3.111595
1     -0.016916 -0.007591      15        1   2.985420
8     -0.023545 -0.028325      16        1   2.911748
3      0.005744 -0.001623      17        1   2.816911
19    -0.017251 -0.009273      18        1   2.678789
13    -0.017260 -0.015527      19        1   2.571455
10    -0.032502 -0.011302      20        1   2.505105, topic_info=     Category       Freq        Term      Total  loglift  logprob
1200  Default   5.000000        just   5.000000  30.0000  30.0000
1255  Default   4.000000         let   4.000000  29.0000  29.0000
680   Default  12.000000   emissions  12.000000  28.0000  28.0000
2171  Default   5.000000     support   5.000000  27.0000  27.0000
1408  Default   2.000000      moment   2.000000  26.0000  26.0000
...       ...        ...         ...        ...      ...      ...
1892  Topic20   0.154448        rest   1.401763   1.4812  -6.0567
208   Topic20   0.168354      better   2.830342   0.8648  -5.9705
930   Topic20   0.170728      global   4.610742   0.3908  -5.9565
941   Topic20   0.170567  government   8.954485  -0.2739  -5.9575
1782  Topic20   0.155154        read   1.683619   1.3026  -6.0522

[1052 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
27        1  0.207320     act
27        2  0.207320     act
27        3  0.207320     act
29        1  0.222789  action
29        5  0.222789  action
...     ...       ...     ...
2467      2  0.128551    zero
2467      3  0.257102    zero
2467      4  0.128551    zero
2467      5  0.128551    zero
2467      7  0.128551    zero

[353 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 18, 3, 17, 12, 8, 16, 6, 1, 15, 7, 13, 5, 10, 2, 9, 4, 20, 14, 11])

In [34]:
lda_tf

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [24]:
lda_tf.methods()

AttributeError: 'LatentDirichletAllocation' object has no attribute 'methods'

In [25]:
dir(lda_tf)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_approx_bound',
 '_check_non_neg_array',
 '_check_params',
 '_e_step',
 '_em_step',
 '_get_param_names',
 '_get_tags',
 '_init_latent_vars',
 '_perplexity_precomp_distr',
 '_unnormalized_transform',
 'batch_size',
 'bound_',
 'components_',
 'doc_topic_prior',
 'doc_topic_prior_',
 'evaluate_every',
 'exp_dirichlet_component_',
 'fit',
 'fit_transform',
 'get_params',
 'learning_decay',
 'learning_method',
 'learning_offset',
 'max_doc_update_iter',
 'max_iter',
 'mean_change_tol',
 'n_batch_iter_',
 'n_components',
 'n_iter_',
 'n_jobs',
 'partial_fit',
 'perp_tol',
 'perplexity',
 'ran

In [34]:
transformed = lda_tf.transform(dtm_tf)
transformed

array([[1.25000002e-02, 1.25000000e-02, 1.25000000e-02, ...,
        1.25000000e-02, 1.25000003e-02, 1.25000004e-02],
       [9.13636362e-01, 4.54545464e-03, 4.54545463e-03, ...,
        4.54545463e-03, 4.54545458e-03, 4.54545512e-03],
       [7.93650816e-04, 7.93650816e-04, 6.67395994e-02, ...,
        7.93650807e-04, 7.93650807e-04, 7.93650814e-04],
       ...,
       [1.61290327e-03, 1.61290324e-03, 1.61290326e-03, ...,
        1.61290325e-03, 1.61290324e-03, 1.61290324e-03],
       [2.63157905e-03, 2.63157898e-03, 2.63157897e-03, ...,
        2.63157902e-03, 2.63157899e-03, 2.63157901e-03],
       [1.78571431e-03, 1.78571431e-03, 1.78571434e-03, ...,
        1.78571431e-03, 1.78571431e-03, 1.78571431e-03]])

In [38]:
transformed[0]
dir(dtm_tf[0])

['__abs__',
 '__add__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmatmul__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_add_dense',
 '_add_sparse',
 '_arg_min_or_max',
 '_arg_min_or_max_axis',
 '_asindices',
 '_binopt',
 '_cs_matrix__get_has_canonical_format',
 '_cs_matrix__get_sorted',
 '_cs_matrix__set_has_canonical_format',
 '_cs_matrix__set_sorted

In [46]:
print(dtm_tf[0].data)

[1 1 1]


In [47]:
raw_texts[0]

'Support to industries such as hemp farming\n'

In [9]:
dir(dtm_tf)

['__abs__',
 '__add__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmatmul__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_add_dense',
 '_add_sparse',
 '_arg_min_or_max',
 '_arg_min_or_max_axis',
 '_asindices',
 '_binopt',
 '_cs_matrix__get_has_canonical_format',
 '_cs_matrix__get_sorted',
 '_cs_matrix__set_has_canonical_format',
 '_cs_matrix__set_sorted

In [12]:
print(dtm_tf[1])

  (0, 2467)	1
  (0, 680)	1
  (0, 340)	1
  (0, 306)	1
  (0, 1250)	1
  (0, 1741)	1
  (0, 1817)	1
  (0, 338)	1
  (0, 1328)	2
  (0, 490)	1
  (0, 941)	1
  (0, 609)	1


In [18]:
for d in dtm_tf:
    print(d.data.real)
    

[1 1 1 1]
[1 1 1 1 1 1 1 1 2 1 1 1]
[2 4 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 1 1 1 1 2 4 6 6 3 2 2 1 1
 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1]
[2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1]
[1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 4 1 2
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 1 1 2 1 1 1]
[1 1 1 1 1 2 1 1 1 1]
[1 3 1 1 2 1 2 1 3 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 6 1 1 3 1 1 1 1 1 2 2 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 2 1 1 1 1]
[1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 4 3 1 3 3 1 1 1 4 1 1 1 2 1 1 1 1 3 3 2 1 1 1 2 1 1 1 1 2 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 2 1 1 1 1 1 1]
[1 1 1 1 1 1 3 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 2
 2 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1]
[1 1 2 1 1]
[1 1 1]
[1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 

In [36]:
tf_vectorizer.get_feature_names()

['ability',
 'able',
 'abroad',
 'absolutely',
 'absorb',
 'absorbed',
 'accelerated',
 'accelerating',
 'accept',
 'acceptable',
 'accepted',
 'accepting',
 'access',
 'accessibility',
 'accessible',
 'accord',
 'according',
 'account',
 'accountable',
 'accounting',
 'accurate',
 'achievable',
 'achieve',
 'achieved',
 'achieving',
 'acidification',
 'acknowledge',
 'act',
 'acting',
 'action',
 'actions',
 'active',
 'actively',
 'activities',
 'activity',
 'actual',
 'actually',
 'adapt',
 'adaptation',
 'adapted',
 'adapting',
 'adaption',
 'add',
 'added',
 'adding',
 'addition',
 'additional',
 'additionally',
 'address',
 'addressed',
 'addressing',
 'adds',
 'adjustment',
 'administrative',
 'adopt',
 'adopted',
 'advantage',
 'advantages',
 'adverse',
 'adversely',
 'advertised',
 'advice',
 'advisors',
 'advisory',
 'advocate',
 'affect',
 'affected',
 'affecting',
 'afford',
 'age',
 'agencies',
 'agenda',
 'ages',
 'aggressively',
 'ago',
 'agree',
 'agreement',
 'agreemen

In [37]:
dtm_tf

<599x2468 sparse matrix of type '<class 'numpy.int64'>'
	with 16990 stored elements in Compressed Sparse Row format>